In [47]:
import pandas as pd

data = pd.read_csv("08_12_2024-08:42PM:23.csv", header=0)
data

,Time [s],State Linear velocity [m/s],State Angular velocity [rad/s],Control Linear velocity [m/s],Control Angular velocity [rad/s]
0,0.630073,0.000,0.000,0.005293,-0.021705
1,0.683112,0.000,0.000,-0.009759,0.009334
2,0.769191,0.000,0.000,-0.025014,0.036144
3,0.829258,0.000,0.000,-0.032400,0.060770
4,0.842225,0.000,0.000,-0.020980,0.071208
...,...,...,...,...,...
1451,66.102048,0.022,-0.055,0.057216,-0.126450
1452,66.131004,0.044,-0.111,0.063635,-0.110019
1453,66.179227,0.057,-0.148,0.042395,-0.096173
1454,66.225588,0.057,-0.148,0.028938,-0.067682


In [48]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=2, cols=1)

fig.add_trace(go.Scatter(x=data["Time [s]"], y=data["State Linear velocity [m/s]"]), row=1, col=1)
fig.add_trace(go.Scatter(x=data["Time [s]"], y=data["Control Linear velocity [m/s]"]), row=1, col=1)

fig.add_trace(go.Scatter(x=data["Time [s]"], y=data["State Angular velocity [rad/s]"]), row=2, col=1)
fig.add_trace(go.Scatter(x=data["Time [s]"], y=data["Control Angular velocity [rad/s]"]), row=2, col=1)

fig.update_layout(height=700)
fig.show()

In [49]:
# Integrate forward
positions = [np.zeros(2)]
thetas = [0.0]
angular_velocity = data["State Angular velocity [rad/s]"].to_numpy()
linear_velocity = data["State Linear velocity [m/s]"].to_numpy()
times = data["Time [s]"]
for i in range(1, len(angular_velocity)):
    dT = times[i] - times[i-1]
    thetas.append(thetas[-1] + angular_velocity[i]*dT)

    velocity_world = np.array([[np.cos(thetas[-1]), -np.sin(thetas[-1])], 
                               [np.sin(thetas[-1]), np.cos(thetas[-1])]])@np.array([linear_velocity[i], 0.0])
    
    positions.append(positions[-1]+velocity_world*dT)

positions = np.array(positions)

fig = make_subplots(rows=2, cols=1)

fig.add_trace(go.Scatter(x=[0.0, 1.5, 1.5, 0.0, 0.0], y=[0.0, 0.0, 1.5, 1.5, 0.0], name="Reference Square", marker_color="black"), row=1, col=1)
fig.add_trace(go.Scatter(x=-positions[:,1], y=positions[:,0], name="Estimated Position", marker_color="blue"), row=1, col=1)
fig.add_trace(go.Scatter(x=times, y=np.rad2deg(thetas), showlegend=False), row=2, col=1)

fig.add_trace(go.Scatter(x=[0.3], y=[-0.3], marker_size=5, name="landing point"), row=1, col=1)

fig.update_layout(width=600, height=1100)
fig.show()

In [50]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=[0.0, 1.5, 1.5, 0.0, 0.0], y=[0.0, 0.0, 1.5, 1.5, 0.0], name="Reference Square", marker_color="black"))
fig.add_trace(go.Scatter(x=-positions[:,1], y=positions[:,0], name="Estimated Position", marker_color="blue"))
fig.add_trace(go.Scatter(x=-positions[-1:,1], y=positions[-1:,0], showlegend=False, marker_color="blue"))

fig.add_trace(go.Scatter(x=[0.3], y=[-0.3], marker_size=14, name="landing point"))

fig.update_layout(xaxis_title="X [m]", yaxis_title="Y [m]",
    width=600, height=500, template="plotly_white")
fig.show()